# NBA Statistics Predictor

## Player List
First we will define the year that we will train the model from <br>
We will then make a table of players that retired after that year

In [5]:
# Install dependencies
%pip install string
%pip install pandas
import string
import pandas as pd
import random
import time

ERROR: Could not find a version that satisfies the requirement string (from versions: none)
ERROR: No matching distribution found for string
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:


year_from = 2023
alphabet = string.ascii_lowercase

# Initialize an empty list to store player DataFrames
all_players = []

# Iterate through each letter in the alphabet to fetch player data
for letter in alphabet:
    random_seconds = random.randint(10, 20)
    print(f"Beginning letter {letter} and sleeping for {random_seconds} seconds")
    time.sleep(random_seconds)
    
    # Format the URL with the current letter
    url = f"https://www.basketball-reference.com/players/{letter}/"
    players_table = pd.read_html(url)[0]
    
    # Filter players based on retirement year
    players_table['To'] = pd.to_numeric(players_table['To'], errors='coerce')  # Convert 'To' column to numeric
    active_players = players_table[players_table['To'] > year_from]
    
    # Append the filtered DataFrame to the list
    all_players.append(active_players)

# Concatenate all player DataFrames into a single DataFrame
all_players_df = pd.concat(all_players, ignore_index=True)

# Save the DataFrame to a CSV file with the correct file name
file_name = f'players_list_{year_from}.csv'
all_players_df.to_csv(file_name, index=False)

print(f"Data saved to {file_name}")

## Individual Player Data

Now there is a csv file in the directory with a list of players that retired after the year selected <br>
<br>
We will now itterate over each player and create a data frame with their game logs for their carrer <br>
The goal is to create a csv file for each all players and the games that they played


In [8]:
# functions for player data pre processing 
def get_player_years_played_url(row):
    player_name = row['Player'].lower().split()
    first_letter_last = player_name[1][0]
    unique_id = player_name[1][:4]
    return f"https://www.basketball-reference.com/players/{first_letter_last}/{unique_id}01.html"
    

def get_player_years_played(player_url):
    time.sleep(random.randint(5, 10))
    tables = pd.read_html(player_url)
    player_seasons = tables[1] #7 for player
    print(player_seasons)
    
    
    
    

In [10]:
# import the list of players
player_list = pd.read_csv('players_list_2023.csv')

num_players = len(player_list)

print(f"staring to proccess player data for {num_players} players")

players_table = pd.DataFrame({})


for index in range(3): # change this for production
    # this is the player
    row = player_list.iloc[index]
    player_url = get_player_years_played_url(row)
    print(player_url)
    years_played = get_player_years_played(player_url)






"""
player_df = player_df.drop(['Rk', 'G'], axis=1)
player_df.rename(columns={'Unnamed: 7': 'PtDiff', 'Unnamed: 5': 'Home/Away',}, inplace=True)


for index, row in df.iterrows():
    print(f"Index: {index}")
    print(f"Name: {row['Name']}, Age: {row['Age']}, City: {row['City']}")
    print()  # Add a blank line for readability

print(player_df.columns)
"""


staring to proccess player data for 568 players
https://www.basketball-reference.com/players/a/achi01.html


HTTPError: HTTP Error 404: Not Found